In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from flask import Flask, jsonify,render_template
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from mysql_conn import password
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()
import requests
import pandas as pd

In [3]:
connection_string = (f"root:{password}@localhost/real_estate")
engine = create_engine(f"mysql://{connection_string}")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Save reference to the table
mean_sales_count = Base.classes.mean_sales_count
median_sales_count = Base.classes.median_sales_count
median_price_sqft = Base.classes.median_price_sqft
median_price_zip = Base.classes.median_price_zip
print(Base.classes.keys())

# Create our connection object
session = Session(engine)

['mean_sales_count', 'median_price_sqft', 'median_price_zip', 'median_sales_count']


In [4]:
city_dict = {}
results = (session.query(mean_sales_count.Metro).all())
city_list = list(np.ravel(results))
print(city_list)

city_ST = {'Austin-Round Rock': 'Austin, TX', 
            'Dallas-Fort Worth-Arlington': 'Dallas-Forth Worth, TX', 
            'Denver-Aurora-Lakewood': 'Denver, CO', 
            'Detroit-Warren-Dearborn': 'Detroit, MI', 
            'New York-Newark-Jersey City': 'New York City, NY', 
            'Orlando-Kissimmee-Sanford': 'Orlando, FL', 
            'Raleigh-Durham-Chapel Hill': 'Raleigh-Durham, NC', 
            'San Francisco-Oakland-Hayward': 'San Francisco, CA', 
            'Seattle-Tacoma-Bellevue': 'Seattle, WA', 
            'Washington-Arlington-Alexandria': 'Washington D.C.'}

['Austin-Round Rock', 'Dallas-Fort Worth-Arlington', 'Denver-Aurora-Lakewood', 'Detroit-Warren-Dearborn', 'New York-Newark-Jersey City', 'Orlando-Kissimmee-Sanford', 'Raleigh-Durham-Chapel Hill', 'San Francisco-Oakland-Hayward', 'Seattle-Tacoma-Bellevue', 'Washington-Arlington-Alexandria']


In [5]:
city_dict = {}
for city in city_list[2:]:
    price_results = ((session.query(median_price_zip._2008,
                                        median_price_zip._2009,
                                        median_price_zip._2010,
                                        median_price_zip._2011,
                                        median_price_zip._2012,
                                        median_price_zip._2013,
                                        median_price_zip._2014,
                                        median_price_zip._2015,
                                        median_price_zip._2016,
                                        median_price_zip._2017,
                                        median_price_zip._2018)).filter(median_price_zip.Metro == city).all())

    price_results_list = list(np.ravel(price_results))
    
    city_dict[city_ST[city]] = {
        "Metro Area": city,
        "Median Sale Price Per Year": {
            "2008": str(price_results_list[0]),
            "2009": str(price_results_list[1]),
            "2010": str(price_results_list[2]),
            "2011": str(price_results_list[3]),
            "2012": str(price_results_list[4]),
            "2013": str(price_results_list[5]),
            "2014": str(price_results_list[6]),
            "2015": str(price_results_list[7]),
            "2016": str(price_results_list[8]),
            "2017": str(price_results_list[9]),
            "2018": str(price_results_list[10])
        }}
    print(city_dict)

{'Denver, CO': {'Metro Area': 'Denver-Aurora-Lakewood', 'Median Sale Price Per Year': {'2008': '2266061.1538461498', '2009': '2636127.3526905398', '2010': '2706614.5038429401', '2011': '2682096.8148520198', '2012': '2833528.0706847799', '2013': '3111792.5511506000', '2014': '3366779.4039293998', '2015': '3777083.2814021399', '2016': '4195967.7028484298', '2017': '4574974.6987951798', '2018': '4980854.1852086503'}}}
{'Denver, CO': {'Metro Area': 'Denver-Aurora-Lakewood', 'Median Sale Price Per Year': {'2008': '2266061.1538461498', '2009': '2636127.3526905398', '2010': '2706614.5038429401', '2011': '2682096.8148520198', '2012': '2833528.0706847799', '2013': '3111792.5511506000', '2014': '3366779.4039293998', '2015': '3777083.2814021399', '2016': '4195967.7028484298', '2017': '4574974.6987951798', '2018': '4980854.1852086503'}}, 'Detroit, MI': {'Metro Area': 'Detroit-Warren-Dearborn', 'Median Sale Price Per Year': {'2008': '1514714.2857142901', '2009': '1497826.9217510500', '2010': '13798

In [7]:
city_dict = {}
results = (session.query(mean_sales_count.Metro).all())
city_list = list(np.ravel(results))
print(city_list)

city_ST = {'Austin-Round Rock': 'Austin, TX', 
            'Dallas-Fort Worth-Arlington': 'Dallas-Forth Worth, TX', 
            'Denver-Aurora-Lakewood': 'Denver, CO', 
            'Detroit-Warren-Dearborn': 'Detroit, MI', 
            'New York-Newark-Jersey City': 'New York City, NY', 
            'Orlando-Kissimmee-Sanford': 'Orlando, FL', 
            'Raleigh-Durham-Chapel Hill': 'Raleigh-Durham, NC', 
            'San Francisco-Oakland-Hayward': 'San Francisco, CA', 
            'Seattle-Tacoma-Bellevue': 'Seattle, WA', 
            'Washington-Arlington-Alexandria': 'Washington D.C.'}

coordinates = [[30.26, -97.74],[32.75, -97.33],[39.73, -104.99],
                    [42.33, -83.04],[40.71, -74.00],[28.53, -81.37],
                    [35.77, -78.63],[37.77, -122.41],[47.60, -122.33],[38.90, -77.03]]

for i in range(10):
    price_results = ((session.query(median_price_zip._2013,
                                        median_price_zip._2014,
                                        median_price_zip._2015,
                                        median_price_zip._2016,
                                        median_price_zip._2017,
                                        median_price_zip._2018)).filter(median_price_zip.Metro == city_list[i]).all())

    price_results_list = list(np.ravel(price_results))
    city_dict.setdefault("locations", [])
    city_dict["locations"].append({
        "Coordinates": coordinates[i],
        "city":{
            "name":city_ST[city_list[i]],
            "Metro Area": city_list[i],
            "Median Sale Price Per Year": {
                "2013": str(price_results_list[0]),
                "2014": str(price_results_list[1]),
                "2015": str(price_results_list[2]),
                "2016": str(price_results_list[3]),
                "2017": str(price_results_list[4]),
                "2018": str(price_results_list[5])
        }}})
print(city_dict)

In [14]:
session.query(median_price_zip._2018).filter(median_price_zip.Metro == city_list[i]).all()

[]